In [81]:
# importing modules
import os # for reading data
import fitz # for conversion of pdf to text

In [82]:
# default directories for reading
books_dir = 'books/'
podcasts_dir = 'podcasts/'

In [83]:
# function to read file
def read_pdf(name):
    """
    Function to read text data from the given pdf file
    Returns the data in form of dictionary:
    key->pg no  value-> text on that page
    """
    # text storage
    data = {}
    # check if the file is present in the directory
    try:
        file_path = os.path.join(books_dir, name)
        # if it exists
        if os.path.exists(file_path):
            # we read the data
            with fitz.open(file_path) as pdf_doc:
                for page_num in range(pdf_doc.page_count):
                    text = pdf_doc.load_page(page_num).get_text()
                    if text:
                        data[page_num+1] = text
        else:
            print(f"The file '{name}' does not exist in the directory '{books_dir}'.")
    except Exception as e:
        print(f"An error occured:{e}")
    finally:
        return data

### Reading file from path ###

In [84]:
# reading files
file_name = "1. Harry Potter and the Sorcerers Stone.pdf"
# reading data from the file
data = read_pdf(file_name)

In [85]:
i = 0
for x, y in enumerate(data):
    print(x, data[y])
    if i == 2:
        break
    i += 1

0 1
Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.

### Taking page numbers as input for the summary generation ###

In [86]:
from_page = int(input())
to_page = int(input())

In [87]:
text = ""
for x, y in enumerate(data):
    if from_page<=x+1<=to_page:
        text += data[y][1:]+" "

In [88]:
print(text)


Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Po

### Summarisation process ###

In [89]:
import openai
api_key = None

with open('openai.api', 'r') as api:
    api_key = api.read()
api.close()

openai.api_key = api_key

In [90]:
tokens = 1024*4
segments = 2*tokens//4

In [91]:
data_pieces = []
# creating data_pieces of segments of 1000 words
for i in range(0, len(text), 1000):
    data_pieces.append(text[i:i+1000])


In [92]:
def summarize(input_data):
    prompt = "Create a summary capturing the main points of the following text:\n\n" + input_data
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=tokens-len(input_data),
    )
    completed = response['choices'][0]['text'].strip()
    return completed

In [93]:
summary = ""
for i in range(len(data_pieces)):
    summary += summarize(summary + data_pieces[i])

In [94]:
print(summary)

- The Dursleys are a normal family who are proud of their normalcy. They have a son, Dudley, and lead normal lives. The only thing abnormal about them is that they are intensely private about the Potters, to the point of fear that someone might find out about them.The Dursleys are a family who pride themselves on their normalcy. The only thing abnormal about them is their intense privacy about the Potters, who are Mrs. Dursley's sister and her husband. The Dursleys are afraid of what the neighbors might say if they found out about the Potters. The Potters have a small son, but the Dursleys have never even seen him. When strange and mysterious things start happening all over the country, the Dursleys are none the wiser.shake and put the cat out of his mind.

The Dursleys are a normal family who pride themselves on their normalcy. The only thing abnormal about them is their intense privacy about the Potters, who are Mrs. Dursley's sister and her husband. The Dursleys are afraid of what t

### For podcast ###

In [95]:
# importing modules
import pvleopard
from pydub import AudioSegment
import os

In [96]:
def get_audio_length(file_path):
    audio = AudioSegment.from_file(file_path)
    length_in_seconds = len(audio) / 1000  # Convert milliseconds to seconds
    return length_in_seconds

In [97]:
def crop_audio(input_mp3, start_time, end_time, cropped_mp3):
    audio = AudioSegment.from_mp3(input_mp3)

    # Calculate the start and end positions in milliseconds
    start_ms = start_time * 1000
    end_ms = end_time * 1000

    # Crop the audio segment
    cropped_audio = audio[start_ms:end_ms]

    # Export the cropped audio to a new MP3 file
    cropped_audio.export(cropped_mp3, format="mp3")
    return cropped_mp3

In [98]:
def audio_to_text(CROPPED_AUDIO):
    leopard = pvleopard.create(access_key="CBt7IGZqr4Ar4y29YdpwSRjvc3XczJUkOLF1+cpyywrbe7ZtjX2Hyg==")
    transcript, words = leopard.process_file(CROPPED_AUDIO)
    return transcript

In [99]:
# transcription preparation process

file_path = r"G:\hackathon\hackathon_proj\Summarisation\podcasts\MBp1.mp3"
# if file not present in the directory
os.path.join(os.getcwd(), 'podcasts')
if not os.path.isfile(file_path):
    print("File not found")
    exit()
else:
    print("File found")
    print("Transcribing audio...")
    start_second = max(0, int(input("Enter start second: ")))
    ouptut_name = "abcd.mp3"
    # audio_len = get_audio_length(file_path)
    end_second = int(input("Enter end second: "))
    cropped_audio = crop_audio(file_path, start_second, end_second, ouptut_name)
    transcript = audio_to_text(cropped_audio)
    with open("transcript.txt", "w", encoding="utf-8") as file:
        file.write(transcript)

File found
Transcribing audio...


Summarising the transcript of a meeting using NLP

In [100]:
# reading the transcript.txt file
data = ""
with open('transcript.txt', 'r') as file:
    data = file.read().replace('\n', '')

summarised_podcast = summarize(data)
# saving summarised text to a file
with open('summarised_podcast.txt', 'w') as file:
    file.write(summarised_podcast)


In [101]:
print(summarised_podcast)

my hope is that you will enjoy this as much as I have enjoyed retelling

This is the first installment of a Mahabharata podcast series. The goal is to retell the ancient Indian epic in an abridged, but not too abridged, audio format. The story is about the war between the cousins and the royal Krishna played in it. The hope is that listeners will enjoy the story as much as the narrator has enjoyed retelling it.


### Converting text to speech

In [102]:
from gtts import gTTS

In [103]:
def text_to_audio(input_file, language='en', output_file='output.mp3'):
    """
    Convert text from a file to audio using Google Text-to-Speech (gTTS) library.

    Parameters:
        input_file (str): The path to the input text file.
        language (str): The language code (default is 'en' for English).
        output_file (str): The name of the output audio file (default is 'output.mp3').

    Returns:
        None
    """
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read().replace('\n', ' ')

    tts = gTTS(text=text, lang=language, slow=False)
    tts.save(output_file)
    print(f"Text from {input_file} converted to audio and saved as {output_file}")

In [104]:
# converting summary to audio
text_to_audio('summarised_podcast.txt')

Text from summarised_podcast.txt converted to audio and saved as output.mp3
